<a href="https://colab.research.google.com/github/maximecharriere/movie-chatbot/blob/master/movie-chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbot cinéphile - WELCOME**
*Par Dylan **Morocutti** et Maxime **Charrière**.*

# Libraries importation

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import treebank
import pandas as pd

print("Libraries version:")
print(f"Numpy:      {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Tensorflow: {tf.__version__}")
print(f"NLTK:       {nltk.__version__}")
print(f"Pandas:     {pd.__version__}")

Libraries version:
Numpy:      1.18.4
Matplotlib: 3.2.1
Tensorflow: 2.2.0
NLTK:       3.2.5
Pandas:     1.0.3


# Importation des data
- Source: http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [0]:
url_movie_line = "https://raw.githubusercontent.com/maximecharriere/movie-chatbot/master/data/parsed_movie_dialogue.txt"
movie_line = pd.read_csv(url_movie_line, sep='\+{3}\$\+{3}', engine='python', names=("First line","Reply"))

In [0]:
movie_line

,First line,Reply
0,"Admiral, we're in position. All fighters accou...",Proceed with the countdown. All groups assume...
1,"All craft, prepare to jump to hyperspace on my...",All right. Stand by.
2,We saw it. All craft prepare to retreat.,"You won't get another chance at this, Admiral."
3,"You won't get another chance at this, Admiral.","We have no choice, General Calrissian. Our cr..."
4,"We have no choice, General Calrissian. Our cru...",Han will have that shield down. We've got to ...
...,...,...
165,Yes or no will do.,"Oh. Well, yes."
166,"Oh. Well, yes.",How many languages do you speak?
167,How many languages do you speak?,I am fluent in over six million forms of comm...
168,I am fluent in over six million forms of commu...,Splendid! We have been without an interpreter...


# Building DeepLearning **model**

In [0]:
# Flatten
model = tf.keras.models.Sequential()
# Add layers
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

# NLTK Test

### Download data

In [0]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [0]:
sentence = """Your insight serves you well. Bury your feelings deep down, Luke. They do you credit. But they could be made to serve the Emperor."""
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
entities = nltk.chunk.ne_chunk(tagged)
# t = treebank.parsed_sents('wsj_0001.mrg')[0]
# nltk.draw.tree.TreeView(t)._cframe.print_to_file('output.ps')

In [0]:
len(tagged)

29